In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
from pathlib import Path
sys.path.append("..")

from scripts.data_utils.eda import FinanceEDA
# from scripts.data_utils.utils import *
from scripts.data_utils.loader import FinanceDataLoader
from scripts.data_utils.cleaner import FinanceDataCleaner

from scripts.modeling.utils import *
from scripts.modeling.arima_model import *
from scripts.modeling.lstm_model import *
from scripts.modeling.sarima_model import *


ModuleNotFoundError: No module named 'scripts.data_utils'

In [ ]:
# -------------------- Data Loading and Preprocessing --------------------
df = pd.read_csv("TSLA.csv", parse_dates=["Date"], index_col="Date")
df

In [ ]:
# Initialize preprocessor and work with 'Close' prices.
preprocessor = TimeSeriesPreprocessor(df, target_col="Close")

In [ ]:
# Test stationarity and make stationary if needed.
preprocessor.test_stationarity()
df_stationary = preprocessor.make_stationary()

In [ ]:
# Scale data for modeling (using MinMaxScaler)
df_scaled, scaler = preprocessor.scale_data()

In [ ]:
# Split the data into training and testing sets.
train, test = preprocessor.split_data(train_size=0.8)

In [ ]:
# -------------------- ARIMA Model Training --------------------
arima_model_instance = ARIMAModel(train["Close"], test["Close"], order=(5,1,0))
arima_model_instance.train_model()

In [ ]:
# Forecast evaluation on test set (for evaluation purpose)
arima_model_instance.evaluate()


In [ ]:
# -------------------- Future Forecasting --------------------
# Define last training date from the training set index.
last_train_date = train.index[-1]

In [ ]:
# Initialize the MarketTrendForecaster with the trained ARIMA model.
trend_forecaster = MarketTrendForecaster(model=arima_model_instance, 
                                        model_name="ARIMA", 
                                        last_train_date=last_train_date, 
                                        scaler=scaler)


In [ ]:
# Forecast future values for the next 6 months (approx. 126 business days)
forecast_steps = 126
forecast_values, conf_intervals = trend_forecaster.forecast_future(forecast_steps=forecast_steps)

In [ ]:
# For plotting, get historical actual prices in original scale.
historical_actual = scaler.inverse_transform(train["Close"].values.reshape(-1, 1))
historical_series = pd.Series(historical_actual.flatten(), index=train.index)

In [ ]:
# Plot the forecast along with historical data and confidence intervals.
trend_forecaster.plot_forecast(historical_data=historical_series, 
                                forecast=forecast_values, 
                                conf_int=conf_intervals,
                                title="TSLA Future Trend Forecast (6 Months)")